# 데이터 가져오기

In [3]:
import yfinance as yf
import pandas as pd
import os

# 다운로드할 종목명 (Yahoo Finance 티커 기준)
TICKERS = {
    'SPY': 'SPY',       # 시장 평균주
    'SCHD': 'SCHD',     # 배당주
    'QQQ': 'QQQ',       # 성장주
    'TLT': 'TLT',       # 장기 국채
    'GLD': 'GLD'        # 금
}

START_DATE = '1998-01-01'
END_DATE = '2025-05-16'
DATA_DIR = './data_raw'
os.makedirs(DATA_DIR, exist_ok=True)

def download_save_ohlcv(ticker_dict, start, end, save_dir):
    for name, yf_ticker in ticker_dict.items():
        df = yf.download(yf_ticker, start=start, end=end)
        if df.empty:
            print(f"❌ {name} 데이터 없음")
            continue

        df.reset_index(inplace=True)
        df['Ticker'] = name
        df.to_csv(os.path.join(save_dir, f'{name}_OHLCV.csv'), index=False)
        print(f"✅ {name} 저장 완료: {name}_OHLCV.csv")

download_save_ohlcv(TICKERS, START_DATE, END_DATE, DATA_DIR)

[*********************100%***********************]  1 of 1 completed


✅ SPY 저장 완료: SPY_OHLCV.csv


[*********************100%***********************]  1 of 1 completed


✅ SCHD 저장 완료: SCHD_OHLCV.csv


[*********************100%***********************]  1 of 1 completed


✅ QQQ 저장 완료: QQQ_OHLCV.csv


[*********************100%***********************]  1 of 1 completed


✅ TLT 저장 완료: TLT_OHLCV.csv


[*********************100%***********************]  1 of 1 completed

✅ GLD 저장 완료: GLD_OHLCV.csv


In [4]:
import yfinance as yf
import pandas as pd
import os

# JEPI 티커만 지정
TICKERS = {'JEPI': 'JEPI'}

START_DATE = '1998-01-01'
END_DATE = '2025-05-16'
DATA_DIR = './data_raw'
os.makedirs(DATA_DIR, exist_ok=True)

def download_save_ohlcv(ticker_dict, start, end, save_dir):
    for name, yf_ticker in ticker_dict.items():
        df = yf.download(yf_ticker, start=start, end=end)
        if df.empty:
            print(f"❌ {name} 데이터 없음")
            continue

        df.reset_index(inplace=True)
        df['Ticker'] = name
        filepath = os.path.join(save_dir, f'{name}_OHLCV.csv')
        df.to_csv(filepath, index=False)
        print(f"✅ {name} 저장 완료: {filepath}")

download_save_ohlcv(TICKERS, START_DATE, END_DATE, DATA_DIR)

[*********************100%***********************]  1 of 1 completed

✅ JEPI 저장 완료: ./data_raw\JEPI_OHLCV.csv


# 월별 리턴 (월간 수익률) 계산해 csv 저장

In [8]:
import pandas as pd
import os
from glob import glob

DATA_DIR = './data_raw'
OUTPUT_DIR = './data_monthly'
os.makedirs(OUTPUT_DIR, exist_ok=True)

file_paths = glob(os.path.join(DATA_DIR, '*_OHLCV.csv'))

for file_path in file_paths:
    name = os.path.basename(file_path).replace('_OHLCV.csv', '')
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df.set_index('Date', inplace=True)
    
    price_col = 'Close'
    monthly_price = df[price_col].resample('ME').last()  # 여기 'ME' 사용
    
    monthly_return = monthly_price.pct_change().dropna()
    
    monthly_return_df = monthly_return.to_frame(name='Monthly Return')
    monthly_return_df.index.name = 'Date'
    
    save_path = os.path.join(OUTPUT_DIR, f'{name}_monthly.csv')
    monthly_return_df.to_csv(save_path)
    print(f'✅ {name} 월간 수익률 저장 완료: {save_path}')

✅ GLD 월간 수익률 저장 완료: ./data_monthly\GLD_monthly.csv
✅ JEPI 월간 수익률 저장 완료: ./data_monthly\JEPI_monthly.csv
✅ QQQ 월간 수익률 저장 완료: ./data_monthly\QQQ_monthly.csv
✅ SCHD 월간 수익률 저장 완료: ./data_monthly\SCHD_monthly.csv
✅ SPY 월간 수익률 저장 완료: ./data_monthly\SPY_monthly.csv
✅ TLT 월간 수익률 저장 완료: ./data_monthly\TLT_monthly.csv


# 월별 가격 (월간 종가) 계산해 csv 저장

In [9]:
import pandas as pd
import os
from glob import glob

DATA_DIR = './data_raw'
OUTPUT_DIR = './data_monthly'
os.makedirs(OUTPUT_DIR, exist_ok=True)

file_paths = glob(os.path.join(DATA_DIR, '*_OHLCV.csv'))

for file_path in file_paths:
    name = os.path.basename(file_path).replace('_OHLCV.csv', '')
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df.set_index('Date', inplace=True)
    
    # 월말 종가 추출 ('ME'는 월말 End, 미래 버전 호환용)
    monthly_price = df['Close'].resample('ME').last()
    monthly_price = monthly_price.dropna()
    
    # DataFrame으로 만들고 이름 설정
    monthly_price_df = monthly_price.to_frame(name='Monthly Close')
    monthly_price_df.index.name = 'Date'
    
    save_path = os.path.join(OUTPUT_DIR, f'{name}_monthly.csv')
    monthly_price_df.to_csv(save_path, float_format='%.4f')
    
    print(f'✅ {name} 월간 종가 저장 완료: {save_path}')

✅ GLD 월간 종가 저장 완료: ./data_monthly\GLD_monthly.csv
✅ JEPI 월간 종가 저장 완료: ./data_monthly\JEPI_monthly.csv
✅ QQQ 월간 종가 저장 완료: ./data_monthly\QQQ_monthly.csv
✅ SCHD 월간 종가 저장 완료: ./data_monthly\SCHD_monthly.csv
✅ SPY 월간 종가 저장 완료: ./data_monthly\SPY_monthly.csv
✅ TLT 월간 종가 저장 완료: ./data_monthly\TLT_monthly.csv


# 배당 데이터 -> csv 저장

In [5]:
import yfinance as yf
import os

# 저장할 폴더명
DIVIDEND_FOLDER = 'dividend_data'
os.makedirs(DIVIDEND_FOLDER, exist_ok=True)

# 관심 종목 리스트
#etfs = ['SPY', 'QQQ', 'TLT', 'GLD', 'SCHD', 'JEPI']
etfs = ['SPY', 'QQQ', 'TLT', 'GLD', 'JEPI']

for symbol in etfs:
    ticker = yf.Ticker(symbol)
    dividends = ticker.dividends  # Series: index = date, value = dividend
    if not dividends.empty:
        df = dividends.reset_index()
        df.columns = ['Date', 'Dividend']
        df.to_csv(f'{DIVIDEND_FOLDER}/{symbol}_dividends.csv', index=False)
        print(f'{symbol} 배당 데이터 저장 완료.')
    else:
        print(f'{symbol} 배당 데이터 없음.')

SPY 배당 데이터 저장 완료.
QQQ 배당 데이터 저장 완료.
TLT 배당 데이터 저장 완료.
GLD 배당 데이터 없음.
JEPI 배당 데이터 저장 완료.


- SPY 배당 데이터 저장 완료.
- QQQ 배당 데이터 저장 완료.
- TLT 배당 데이터 저장 완료.
- SCHD 배당 데이터 저장 완료.
- JEPI 배당 데이터 저장 완료.
<br>

- GLD 배당 데이터 없음.

## SCHD -> 연 4회 배당 아님. 잘못된 데이터.

### 아래 사이트에서 SCHD Historical Dividends 확인해 txt로 저장
- -> CSV로 변환하기 전에, 날짜 형식 변경 필요
- https://www.dividendchannel.com/history/?symbol=schd

### 아래 공식 사이트의 정보와 비교해 확인
- https://www.schwabassetmanagement.com/products/schd
- (distributions)

In [4]:
import pandas as pd

# 헤더가 있는 파일을 정상적으로 읽기
df = pd.read_csv('schd_div_txt.txt', sep='\t', header=0)

# 날짜 형식을 명시적으로 변환
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%y")

# 보기 좋은 포맷으로 변경
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# 저장
df.to_csv("schd_dividends_cleaned.csv", index=False)

print("완료: schd_dividends_cleaned.csv")

완료: schd_dividends_cleaned.csv


#  배당 재투자 반영 누적 수익률 계산해 csv 저장

In [15]:
import os
import pandas as pd

# 폴더 경로
data_raw_path = 'data_raw'
monthly_return_path = 'monthly_return'
monthly_price_path = 'data_monthly'
dividend_path_base = 'dividend_data'

# 처리할 티커 목록에 GLD 추가
tickers = ['SPY', 'QQQ', 'SCHD', 'JEPI', 'TLT', 'GLD']

def calculate_total_return(ticker):
    # 파일 경로 설정
    return_file = os.path.join(monthly_return_path, f'{ticker}_monthly_return.csv')
    price_file = os.path.join(monthly_price_path, f'{ticker}_monthly.csv')
    dividend_file = os.path.join(dividend_path_base, f'{ticker}_dividends.csv')

    # 월간 수익률 데이터 불러오기
    returns = pd.read_csv(return_file)
    returns['Date'] = pd.to_datetime(returns['Date'])
    returns['Date'] = returns['Date'].dt.to_period('M')

    # 월간 종가 데이터 불러오기
    prices = pd.read_csv(price_file)
    prices['Date'] = pd.to_datetime(prices['Date'])
    prices['Date'] = prices['Date'].dt.to_period('M')

    # merge
    df = pd.merge(returns, prices, on='Date', how='inner')

    # 배당 데이터 처리 - GLD는 배당 데이터가 없음
    if os.path.exists(dividend_file):
        dividends = pd.read_csv(dividend_file)

        dividends['Date'] = pd.to_datetime(dividends['Date'], errors='coerce', utc=True)
        dividends = dividends.dropna(subset=['Date'])
        dividends['Date'] = dividends['Date'].dt.tz_localize(None)
        dividends['YearMonth'] = dividends['Date'].dt.to_period('M')

        monthly_div = dividends.groupby('YearMonth')['Dividend'].sum().reset_index()
        monthly_div.rename(columns={'YearMonth': 'Date'}, inplace=True)

        df = pd.merge(df, monthly_div, on='Date', how='left')
        df['Dividend'] = df['Dividend'].fillna(0)

        df['Dividend Yield'] = df['Dividend'] / df['Monthly Close']
    else:
        # 배당 데이터가 없으면 배당 수익률은 0으로 처리
        df['Dividend Yield'] = 0.0

    # 총 수익률 = 월간 수익률 + 배당 수익률
    df['Total Return'] = df['Monthly Return'] + df['Dividend Yield']

    # 누적 수익률 계산
    df['Cumulative Return'] = (1 + df['Total Return']).cumprod()

    # 결과 저장
    save_path = f'total_return/{ticker}_total_return.csv'
    os.makedirs('total_return', exist_ok=True)
    df.to_csv(save_path, index=False)

# 전체 실행
for ticker in tickers:
    calculate_total_return(ticker)